In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import random
import math
import re
import matplotlib.colors
import plotly.express as px
import missingno as msno
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import matplotlib.image as mpimg
import matplotlib.pylab as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import statsmodels.api as sm
import warnings

warnings.filterwarnings("ignore")

import xgboost as xgb

from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import CatBoostRegressor

In [3]:
pd.set_option('display.max_columns',None)

train_data = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv', parse_dates=["startdate"])
test_data = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'

In [4]:
train_data.head(10)

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-tmp2m-14d__tmp2m,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010

In [5]:
from sklearn.impute import SimpleImputer

def impute_number_col(df):
    null_col = ['nmme0-tmp2m-34w__ccsm30',
    'nmme-tmp2m-56w__ccsm3',
    'nmme-prate-34w__ccsm3',
    'nmme0-prate-56w__ccsm30',
    'nmme0-prate-34w__ccsm30',
    'nmme-prate-56w__ccsm3',
    'nmme-tmp2m-34w__ccsm3',
    'ccsm30']
    number_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    fixed_column_df = number_imputer.fit_transform(df[null_col])
    df[null_col] = fixed_column_df
    return df

train_data = impute_number_col(train_data)
test_data = impute_number_col(test_data)

from sklearn.preprocessing import LabelEncoder

def categorical_encode(train_data, test_data):
    le = LabelEncoder()
    train_data['climateregions__climateregion'] = le.fit_transform(train_data['climateregions__climateregion'])
    test_data['climateregions__climateregion'] = le.transform(test_data['climateregions__climateregion'])
    return train_data, test_data

train_data, test_data = categorical_encode(train_data,test_data)

def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

def location_nom(train, test):
    # Ref: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    return train, test

def fill_na(df):
    col_na = ['nmme0-tmp2m-34w__ccsm30', 'nmme-tmp2m-56w__ccsm3', 'nmme-prate-34w__ccsm3', 'nmme0-prate-56w__ccsm30', 
                'nmme0-prate-34w__ccsm30', 'nmme-prate-56w__ccsm3', 'nmme-tmp2m-34w__ccsm3', 'ccsm30']

    g_means =  ['nmme0-tmp2m-34w__nmme0mean', 'nmme-tmp2m-56w__nmmemean', 'nmme-prate-34w__nmmemean', 'nmme0-prate-56w__nmme0mean', 
                'nmme0-prate-34w__nmme0mean', 'nmme-prate-56w__nmmemean', 'nmme-tmp2m-34w__nmmemean', 'nmme0mean']

    g_1 = ['nmme0-tmp2m-34w__cancm30', 'nmme0-tmp2m-34w__cancm40', 'nmme0-tmp2m-34w__ccsm40', 'nmme0-tmp2m-34w__cfsv20',
            'nmme0-tmp2m-34w__gfdlflora0', 'nmme0-tmp2m-34w__gfdlflorb0', 'nmme0-tmp2m-34w__gfdl0', 'nmme0-tmp2m-34w__nasa0']

    g_2 = ['nmme-tmp2m-56w__cancm3', 'nmme-tmp2m-56w__cancm4', 'nmme-tmp2m-56w__ccsm4', 'nmme-tmp2m-56w__cfsv2',
        'nmme-tmp2m-56w__gfdl', 'nmme-tmp2m-56w__gfdlflora', 'nmme-tmp2m-56w__gfdlflorb', 'nmme-tmp2m-56w__nasa']

    g_3 = ['nmme-prate-34w__cancm3', 'nmme-prate-34w__cancm4', 'nmme-prate-34w__ccsm4', 'nmme-prate-34w__cfsv2', 
        'nmme-prate-34w__gfdl', 'nmme-prate-34w__gfdlflora', 'nmme-prate-34w__gfdlflorb', 'nmme-prate-34w__nasa']

    g_4 = [ 'nmme0-prate-56w__cancm30', 'nmme0-prate-56w__cancm40', 'nmme0-prate-56w__ccsm40', 'nmme0-prate-56w__cfsv20',
        'nmme0-prate-56w__gfdlflora0', 'nmme0-prate-56w__gfdlflorb0', 'nmme0-prate-56w__gfdl0', 'nmme0-prate-56w__nasa0']

    g_5 = ['nmme0-prate-34w__cancm30', 'nmme0-prate-34w__cancm40', 'nmme0-prate-34w__ccsm40', 'nmme0-prate-34w__cfsv20',
        'nmme0-prate-34w__gfdlflora0', 'nmme0-prate-34w__gfdlflorb0', 'nmme0-prate-34w__gfdl0', 'nmme0-prate-34w__nasa0']

    g_6 = ['nmme-prate-56w__cancm3', 'nmme-prate-56w__cancm4', 'nmme-prate-56w__ccsm4', 'nmme-prate-56w__cfsv2',
        'nmme-prate-56w__gfdl', 'nmme-prate-56w__gfdlflora', 'nmme-prate-56w__gfdlflorb', 'nmme-prate-56w__nasa']

    g_7 = ['nmme-tmp2m-34w__cancm3', 'nmme-tmp2m-34w__cancm4', 'nmme-tmp2m-34w__ccsm4', 'nmme-tmp2m-34w__cfsv2', 
        'nmme-tmp2m-34w__gfdl','nmme-tmp2m-34w__gfdlflora', 'nmme-tmp2m-34w__gfdlflorb', 'nmme-tmp2m-34w__nasa']

    g_8 = ['cancm30', 'cancm40', 'ccsm40', 'cfsv20', 'gfdlflora0', 'gfdlflorb0', 'gfdl0', 'nasa0']

    gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7, g_8]
    zip_cols = zip(col_na, gs, g_means)
    for c, g, m in tqdm(zip_cols):
        df[c] = (df[m]*9) - df[g].sum(1)
    return df

def categorical_encode(train, test, cat_cols):
    le = LabelEncoder()
    for col in cat_cols:
        train[col] = le.fit_transform(train[[col]])
        test[col] = le.transform(test[[col]])
    return train, test

def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day'] = df['startdate'].dt.day
    return df

def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

def feature_scale(train, test):
    scaler = MinMaxScaler()
    train_ = scaler.fit_transform(train)
    test_ = scaler.transform(test)
    train = pd.DataFrame(train_, columns = train.columns)
    test = pd.DataFrame(test_, columns = test.columns)
    return train, test

def feature_engineering(train_data, test_data):
    train, test = location_nom(train_data, test_data)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    train, test = categorical_encode(train, test, cat_cols=['climateregions__climateregion'])
    
    to_drop = identify_correlated(train.copy(), threshold=.85)
    drop_cols = ['index', 'startdate', target]
    drop_cols.extend(to_drop)
    features = [col for col in train.columns if col not in drop_cols]
    
    X = train[features]
    y = train[target]
    X_test = test[features]
    X, X_test = feature_scale(X, X_test)
    
    return X, y, X_test

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X, y, X_test = feature_engineering(train_data, test_data)
    
X_train, X_val, y_train, y_val = train_test_split(X,y,  test_size=0.33, random_state=42)

0it [00:00, ?it/s]

In [7]:
cb_params = {'iterations': 15000,
            'learning_rate': 0.01,
            'depth': 6,
            'l2_leaf_reg': 3,
            'bagging_temperature': 0,
            'border_count': 128,
            'loss_function': 'RMSE',
            'random_seed': 42,
            'task_type': 'GPU',
            'verbose': 250}

model = CatBoostRegressor(**cb_params)
model.fit(X_train, y_train,  eval_set=(X_val, y_val))

y_pred_catboost = model.predict(X_test)

0:	learn: 9.7853864	test: 9.7785774	best: 9.7785774 (0)	total: 64.7ms	remaining: 16m 11s
250:	learn: 2.2788529	test: 2.2869963	best: 2.2869963 (250)	total: 13.4s	remaining: 13m 5s
500:	learn: 1.6503967	test: 1.6542590	best: 1.6542590 (500)	total: 25.3s	remaining: 12m 11s
750:	learn: 1.4705084	test: 1.4737325	best: 1.4737325 (750)	total: 36.8s	remaining: 11m 38s
1000:	learn: 1.3504181	test: 1.3540417	best: 1.3540417 (1000)	total: 49.9s	remaining: 11m 37s
1250:	learn: 1.2576026	test: 1.2616741	best: 1.2616741 (1250)	total: 1m 3s	remaining: 11m 34s
1500:	learn: 1.1822902	test: 1.1868482	best: 1.1868482 (1500)	total: 1m 18s	remaining: 11m 48s
1750:	learn: 1.1175666	test: 1.1227690	best: 1.1227690 (1750)	total: 1m 30s	remaining: 11m 26s
2000:	learn: 1.0624480	test: 1.0679763	best: 1.0679763 (2000)	total: 1m 43s	remaining: 11m 13s
2250:	learn: 1.0169704	test: 1.0229388	best: 1.0229388 (2250)	total: 1m 56s	remaining: 10m 59s
2500:	learn: 0.9783338	test: 0.9848691	best: 0.9848691 (2500)	total:

In [8]:
# wids_submit = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
#                                tree_method = 'gpu_hist',
#                                n_estimators=15000,
#                                early_stopping_rounds=50,
#                                objective='reg:squarederror',
#                                max_depth=3,random_state = 42,
#                                learning_rate=0.01,gpu_id=0)

# wids_submit.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_val, y_val)], verbose=500)

# y_pred_xgboost = wids_submit.predict(X_test)

In [9]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error as mse

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'max_depth': 8,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 50,
    'min_data_in_leaf': 100,
    'subsample_for_bin': 200000,
    'n_estimators': 15000,
    'early_stopping_rounds': 50,
    'device_type': 'gpu'
}

# Create the LightGBM model object
reg_lgb = lgb.LGBMRegressor(**params)

# Fit the model to the training data
reg_lgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=250)


# Get feature importances
importance_scores = reg_lgb.feature_importances_
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': importance_scores})

# Sort the features by importance score
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Output feature importances to a CSV file
feature_importances.to_csv("lgbm_feature_importances.csv", index=False)

# Generate predictions on the test data
y_pred_lgb = reg_lgb.predict(X_test)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[250]	training's rmse: 1.15326	valid_1's rmse: 1.16472
[500]	training's rmse: 0.887852	valid_1's rmse: 0.902125
[750]	training's rmse: 0.760869	valid_1's rmse: 0.77826
[1000]	training's rmse: 0.68223	valid_1's rmse: 0.702548
[1250]	training's rmse: 0.624664	valid_1's rmse: 0.647629
[1500]	training's rmse: 0.576063	valid_1's rmse: 0.60116
[1750]	training's rmse: 0.53957	valid_1's rmse: 0.566

In [10]:
ensemble_preds = y_pred_catboost*0.6 + y_pred_lgb*0.4

In [11]:
submission = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')
submission[target] = ensemble_preds

submission.to_csv('submission', 
                  index = False) # Set index to false to avoid issues in evaluation

display(submission)

,contest-tmp2m-14d__tmp2m,index
0,28.381671,375734
1,28.566923,375735
2,28.568608,375736
3,28.702891,375737
4,28.808934,375738
...,...,...
31349,5.953743,407083
31350,6.469481,407084
31351,5.680195,407085
31352,6.291400,407086
